In [3]:
import pickle
import pandas as pd
import os
import setup

import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm.notebook import tqdm, trange
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import wandb
from torch.utils.data.sampler import SubsetRandomSampler
import pprint
import matplotlib.pyplot as plt
import torch

from sklearn.preprocessing import MinMaxScaler
import math
from torch.profiler import profile, record_function, ProfilerActivity

from operator import itemgetter
import operator
from random import randint
# from rnn_classes import Dog, DogInput, Race, Races, GRUNet, smallGRUNet, smalll_lin_GRUNet, smalll_prelin_GRUNet
import rnn_tools.rnn_classes as rnn_classes
from rnn_tools.raceDB import build_dataset
import importlib
import datetime
from rnn_tools.model_saver import model_saver, model_saver_wandb
# import rnn_tools.training_testing_gru as training_testing_gru
from torch.nn.utils.rnn import pack_padded_sequence, pack_sequence, pad_packed_sequence,pad_sequence, unpack_sequence, unpad_sequence
import rnn_tools.training_testing_gru_double as training_testing_gru_double
from goto_conversion import goto_conversion

import rnn_tools.training_testing_gru_extra_data as training_testing_gru_extra_data
import rnn_tools.training_testing_gru_extra_data_embedding as training_testing_gru_extra_data_embedding

In [4]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc.
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

import os

os.environ["WANDB_NOTEBOOK_NAME"] = "GRU - v8.0.0 Topaz.ipynb"

Running on the GPU


In [5]:
torch.set_default_tensor_type(torch.FloatTensor)

c:\Users\Nick\.conda\envs\python311\Lib\site-packages\torch\__init__.py:614: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\tensor\python_tensor.cpp:453.)
  _C._set_default_tensor_type(t)


In [6]:
torch.__version__

'2.1.1'

In [7]:
data_file = './data/topaz_data_w_bsp_new3.fth'
df = pd.read_feather(data_file)

In [8]:
for c in df.columns:
    print(c)

meetingDate
state
track
distance
raceId
raceTypeCode
raceNumber
boxNumber
rugNumber
runId
dogId
dogName
weightInKg
sex
trainerId
trainerState
damId
damName
sireId
sireName
win
place
resultTime
resultMargin
resultMarginLengths
dogAgeScaled
startPrice
weightInKgScaled
rolling_box_win_percentage
hasEntryBoxNumberPlus1
hasEntryBoxNumberMinus1
dog_distance_mean_1
dog_boxNumber_mean_1
dog_runTimeNorm_mean_1
dog_place_mean_1
dog_resultMargin_mean_1
dog_split_time_margin_mean_1
dog_split_runTimeNorm_mean_1
dog_time_1_mean_1
dog_run_home_TimeNorm_mean_1
dog_finishingPlaceMovement_mean_1
dog_averageSpeed_mean_1
dog_win_mean_1
dog_distance_mean_365D
dog_boxNumber_mean_365D
dog_runTimeNorm_mean_365D
dog_place_mean_365D
dog_resultMargin_mean_365D
dog_split_time_margin_mean_365D
dog_split_runTimeNorm_mean_365D
dog_time_1_mean_365D
dog_run_home_TimeNorm_mean_365D
dog_finishingPlaceMovement_mean_365D
dog_averageSpeed_mean_365D
dog_win_mean_365D
trainer_distance_mean_1
trainer_boxNumber_mean_1
trainer_

In [9]:
os.getcwd()
import rnn_tools.raceDB
importlib.reload(rnn_tools.raceDB)
# os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA")                                                                                                                                                                                                                                                                                                                
date = datetime.datetime.strptime("2019-01-01", "%Y-%m-%d").date()
hidden_size = 32
states = ["NZ","SA","WA"]
# states = ["VIC"]
# states = ["NSW"]
states = ["NZ"]
states = ["SA"]
# states = ['VIC',  "SA"]
extra_tracks = ["Wentworth Park", "Wagga", "Temora"]
track = ['Murray Bridge Straight']
data_file = './data/gru_inputs_kitchen_sink_new_not_simple.fth'
# data_file = './data/gru_inputs_kitchen_sink_april.fth'
data_file = './data/topaz_data_w_bsp_new3.fth'
# data_file = './data/topaz_data_w_bsp_new_w_price.fth'
# data_file = './data/topaz_data_w_bsp.fth'
# data_file = './data/gru_inputs_kitchen_sink_topaz.fth'
raceDB = rnn_tools.raceDB.build_dataset_topaz(data_file, hidden_size,state_filter=states,date_filter=date, track_filter=track,margin_type='sftmin',v6=True,show_stats=False)
# raceDB = rnn_tools.raceDB.build_dataset_topaz(data_file, hidden_size,state_filter=states,track_filter=extra_tracks,date_filter=date, margin_type='sftmin',v6=True,show_stats=False)
raceDB.create_new_weights_v2()

c:\Users\Nick\Documents\GitHub\grvmodel\python
(2504581, 145)
54
Latest date = 2024-05-06 00:00:00
size after state filter (2504581, 147)
(2504581, 147)
Index(['index', 'meetingDate', 'state', 'track_name', 'dist', 'raceId',
       'race_grade', 'race_num', 'box', 'rugNumber',
       ...
       'prev_race_state', 'next_race', 'stats_topaz', 'dogid', 'raceid',
       'stats_cols', 'track_hash', 'stats', 'race_time', 'stats_cuda'],
      dtype='object', length=148)
state  track_name            
SA     Murray Bridge Straight    13096
Name: count, dtype: int64
Latest date = 2024-04-30 00:00:00
stats = [ 1.00000000e+00  1.00000000e+00  1.00000000e+00  0.00000000e+00
  0.00000000e+00 -1.00000000e+00  3.00000000e+02  7.00000000e+00
  4.57918614e-01  2.00000000e+00  2.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  4.57918614e-01 -2.00000000e+00
  1.75644035e+01  0.00000000e+00  3.97814392e+02  4.14261341e+00
  4.37547475e-01  4.25662899e+00  7.08025551e+00  1.75884470e-01
  4.0

100%|██████████| 25/25 [00:00<?, ?it/s]


race_ids=['479136143', '479136144', '479136145', '479136147', '479136146', '479136148', '479136150', '479136149', '479136152', '479136151', '485632394', '485632399', '485632395', '485632396', '485632397', '485632398', '485632404', '485632402', '485632400', '485632401', '485632403', '485632405', '491085794', '491085799', '491085795', '491085796', '491085797', '491085798', '491085801', '491085804', '491085802', '491085800', '491085803', '491085805', '501809163', '501809167', '501809164', '501809165', '501809166', '501809168', '501809169', '501809170', '501809171', '501809172', '501809173', '516002041', '516002040', '516002044', '516002042', '516002043', '516002047', '516002045', '516002046', '516002048', '516002049', '519461908', '519461907', '519461911', '519461909', '519461910', '519461916', '519461914', '519461913', '519461912', '519461915', '523459917', '523459915', '523459919', '523459916', '523459918', '523459921', '523459922', '523459920', '523459924', '523459923', '525253195', '5

In [10]:
feature_cols = ['dam_finishingPlaceMovement_max_28D',
 'dam_finishingPlaceMovement_max_365D',
 'dam_finishingPlaceMovement_max_91D',
 'dam_finishingPlaceMovement_mean_28D',
 'dam_finishingPlaceMovement_mean_365D',
 'dam_finishingPlaceMovement_mean_91D',
 'dam_finishingPlaceMovement_median_28D',
 'dam_finishingPlaceMovement_median_365D',
 'dam_finishingPlaceMovement_median_91D',
 'dam_finishingPlaceMovement_min_28D',
 'dam_finishingPlaceMovement_min_365D',
 'dam_finishingPlaceMovement_min_91D',
 'dam_finishingPlaceMovement_std_28D',
 'dam_finishingPlaceMovement_std_365D',
 'dam_finishingPlaceMovement_std_91D',
 'dam_marginLog_max_28D',
 'dam_marginLog_max_365D',
 'dam_marginLog_max_91D',
 'dam_marginLog_mean_28D',
 'dam_marginLog_mean_365D',
 'dam_marginLog_mean_91D',
 'dam_marginLog_median_28D',
 'dam_marginLog_median_365D',
 'dam_marginLog_median_91D',
 'dam_marginLog_min_28D',
 'dam_marginLog_min_365D',
 'dam_marginLog_min_91D',
 'dam_marginLog_std_28D',
 'dam_marginLog_std_365D',
 'dam_marginLog_std_91D',
 'dam_placeLog_max_28D',
 'dam_placeLog_max_365D',
 'dam_placeLog_max_91D',
 'dam_placeLog_mean_28D',
 'dam_placeLog_mean_365D',
 'dam_placeLog_mean_91D',
 'dam_placeLog_median_28D',
 'dam_placeLog_median_365D',
 'dam_placeLog_median_91D',
 'dam_placeLog_min_28D',
 'dam_placeLog_min_365D',
 'dam_placeLog_min_91D',
 'dam_placeLog_std_28D',
 'dam_placeLog_std_365D',
 'dam_placeLog_std_91D',
 'dam_prizemoneyLog_max_28D',
 'dam_prizemoneyLog_max_365D',
 'dam_prizemoneyLog_max_91D',
 'dam_prizemoneyLog_mean_28D',
 'dam_prizemoneyLog_mean_365D',
 'dam_prizemoneyLog_mean_91D',
 'dam_prizemoneyLog_median_28D',
 'dam_prizemoneyLog_median_365D',
 'dam_prizemoneyLog_median_91D',
 'dam_prizemoneyLog_min_28D',
 'dam_prizemoneyLog_min_365D',
 'dam_prizemoneyLog_min_91D',
 'dam_prizemoneyLog_std_28D',
 'dam_prizemoneyLog_std_365D',
 'dam_prizemoneyLog_std_91D',
 'dam_runTimeNorm_max_28D',
 'dam_runTimeNorm_max_365D',
 'dam_runTimeNorm_max_91D',
 'dam_runTimeNorm_mean_28D',
 'dam_runTimeNorm_mean_365D',
 'dam_runTimeNorm_mean_91D',
 'dam_runTimeNorm_median_28D',
 'dam_runTimeNorm_median_365D',
 'dam_runTimeNorm_median_91D',
 'dam_runTimeNorm_min_28D',
 'dam_runTimeNorm_min_365D',
 'dam_runTimeNorm_min_91D',
 'dam_runTimeNorm_std_28D',
 'dam_runTimeNorm_std_365D',
 'dam_runTimeNorm_std_91D',
 'dog_finishingPlaceMovement_max_28D',
 'dog_finishingPlaceMovement_max_365D',
 'dog_finishingPlaceMovement_max_91D',
 'dog_finishingPlaceMovement_mean_28D',
 'dog_finishingPlaceMovement_mean_365D',
 'dog_finishingPlaceMovement_mean_91D',
 'dog_finishingPlaceMovement_median_28D',
 'dog_finishingPlaceMovement_median_365D',
 'dog_finishingPlaceMovement_median_91D',
 'dog_finishingPlaceMovement_min_28D',
 'dog_finishingPlaceMovement_min_365D',
 'dog_finishingPlaceMovement_min_91D',
 'dog_finishingPlaceMovement_std_28D',
 'dog_finishingPlaceMovement_std_365D',
 'dog_finishingPlaceMovement_std_91D',
 'dog_marginLog_max_28D',
 'dog_marginLog_max_365D',
 'dog_marginLog_max_91D',
 'dog_marginLog_mean_28D',
 'dog_marginLog_mean_365D',
 'dog_marginLog_mean_91D',
 'dog_marginLog_median_28D',
 'dog_marginLog_median_365D',
 'dog_marginLog_median_91D',
 'dog_marginLog_min_28D',
 'dog_marginLog_min_365D',
 'dog_marginLog_min_91D',
 'dog_marginLog_std_28D',
 'dog_marginLog_std_365D',
 'dog_marginLog_std_91D',
 'dog_placeLog_max_28D',
 'dog_placeLog_max_365D',
 'dog_placeLog_max_91D',
 'dog_placeLog_mean_28D',
 'dog_placeLog_mean_365D',
 'dog_placeLog_mean_91D',
 'dog_placeLog_median_28D',
 'dog_placeLog_median_365D',
 'dog_placeLog_median_91D',
 'dog_placeLog_min_28D',
 'dog_placeLog_min_365D',
 'dog_placeLog_min_91D',
 'dog_placeLog_std_28D',
 'dog_placeLog_std_365D',
 'dog_placeLog_std_91D',
 'dog_prizemoneyLog_max_28D',
 'dog_prizemoneyLog_max_365D',
 'dog_prizemoneyLog_max_91D',
 'dog_prizemoneyLog_mean_28D',
 'dog_prizemoneyLog_mean_365D',
 'dog_prizemoneyLog_mean_91D',
 'dog_prizemoneyLog_median_28D',
 'dog_prizemoneyLog_median_365D',
 'dog_prizemoneyLog_median_91D',
 'dog_prizemoneyLog_min_28D',
 'dog_prizemoneyLog_min_365D',
 'dog_prizemoneyLog_min_91D',
 'dog_prizemoneyLog_std_28D',
 'dog_prizemoneyLog_std_365D',
 'dog_prizemoneyLog_std_91D',
 'dog_runTimeNorm_max_28D',
 'dog_runTimeNorm_max_365D',
 'dog_runTimeNorm_max_91D',
 'dog_runTimeNorm_mean_28D',
 'dog_runTimeNorm_mean_365D',
 'dog_runTimeNorm_mean_91D',
 'dog_runTimeNorm_median_28D',
 'dog_runTimeNorm_median_365D',
 'dog_runTimeNorm_median_91D',
 'dog_runTimeNorm_min_28D',
 'dog_runTimeNorm_min_365D',
 'dog_runTimeNorm_min_91D',
 'dog_runTimeNorm_std_28D',
 'dog_runTimeNorm_std_365D',
 'dog_runTimeNorm_std_91D',
 'sire_finishingPlaceMovement_max_28D',
 'sire_finishingPlaceMovement_max_365D',
 'sire_finishingPlaceMovement_max_91D',
 'sire_finishingPlaceMovement_mean_28D',
 'sire_finishingPlaceMovement_mean_365D',
 'sire_finishingPlaceMovement_mean_91D',
 'sire_finishingPlaceMovement_median_28D',
 'sire_finishingPlaceMovement_median_365D',
 'sire_finishingPlaceMovement_median_91D',
 'sire_finishingPlaceMovement_min_28D',
 'sire_finishingPlaceMovement_min_365D',
 'sire_finishingPlaceMovement_min_91D',
 'sire_finishingPlaceMovement_std_28D',
 'sire_finishingPlaceMovement_std_365D',
 'sire_finishingPlaceMovement_std_91D',
 'sire_marginLog_max_28D',
 'sire_marginLog_max_365D',
 'sire_marginLog_max_91D',
 'sire_marginLog_mean_28D',
 'sire_marginLog_mean_365D',
 'sire_marginLog_mean_91D',
 'sire_marginLog_median_28D',
 'sire_marginLog_median_365D',
 'sire_marginLog_median_91D',
 'sire_marginLog_min_28D',
 'sire_marginLog_min_365D',
 'sire_marginLog_min_91D',
 'sire_marginLog_std_28D',
 'sire_marginLog_std_365D',
 'sire_marginLog_std_91D',
 'sire_placeLog_max_28D',
 'sire_placeLog_max_365D',
 'sire_placeLog_max_91D',
 'sire_placeLog_mean_28D',
 'sire_placeLog_mean_365D',
 'sire_placeLog_mean_91D',
 'sire_placeLog_median_28D',
 'sire_placeLog_median_365D',
 'sire_placeLog_median_91D',
 'sire_placeLog_min_28D',
 'sire_placeLog_min_365D',
 'sire_placeLog_min_91D',
 'sire_placeLog_std_28D',
 'sire_placeLog_std_365D',
 'sire_placeLog_std_91D',
 'sire_prizemoneyLog_max_28D',
 'sire_prizemoneyLog_max_365D',
 'sire_prizemoneyLog_max_91D',
 'sire_prizemoneyLog_mean_28D',
 'sire_prizemoneyLog_mean_365D',
 'sire_prizemoneyLog_mean_91D',
 'sire_prizemoneyLog_median_28D',
 'sire_prizemoneyLog_median_365D',
 'sire_prizemoneyLog_median_91D',
 'sire_prizemoneyLog_min_28D',
 'sire_prizemoneyLog_min_365D',
 'sire_prizemoneyLog_min_91D',
 'sire_prizemoneyLog_std_28D',
 'sire_prizemoneyLog_std_365D',
 'sire_prizemoneyLog_std_91D',
 'sire_runTimeNorm_max_28D',
 'sire_runTimeNorm_max_365D',
 'sire_runTimeNorm_max_91D',
 'sire_runTimeNorm_mean_28D',
 'sire_runTimeNorm_mean_365D',
 'sire_runTimeNorm_mean_91D',
 'sire_runTimeNorm_median_28D',
 'sire_runTimeNorm_median_365D',
 'sire_runTimeNorm_median_91D',
 'sire_runTimeNorm_min_28D',
 'sire_runTimeNorm_min_365D',
 'sire_runTimeNorm_min_91D',
 'sire_runTimeNorm_std_28D',
 'sire_runTimeNorm_std_365D',
 'sire_runTimeNorm_std_91D',
 'trainer_finishingPlaceMovement_max_28D',
 'trainer_finishingPlaceMovement_max_365D',
 'trainer_finishingPlaceMovement_max_91D',
 'trainer_finishingPlaceMovement_mean_28D',
 'trainer_finishingPlaceMovement_mean_365D',
 'trainer_finishingPlaceMovement_mean_91D',
 'trainer_finishingPlaceMovement_median_28D',
 'trainer_finishingPlaceMovement_median_365D',
 'trainer_finishingPlaceMovement_median_91D',
 'trainer_finishingPlaceMovement_min_28D',
 'trainer_finishingPlaceMovement_min_365D',
 'trainer_finishingPlaceMovement_min_91D',
 'trainer_finishingPlaceMovement_std_28D',
 'trainer_finishingPlaceMovement_std_365D',
 'trainer_finishingPlaceMovement_std_91D',
 'trainer_marginLog_max_28D',
 'trainer_marginLog_max_365D',
 'trainer_marginLog_max_91D',
 'trainer_marginLog_mean_28D',
 'trainer_marginLog_mean_365D',
 'trainer_marginLog_mean_91D',
 'trainer_marginLog_median_28D',
 'trainer_marginLog_median_365D',
 'trainer_marginLog_median_91D',
 'trainer_marginLog_min_28D',
 'trainer_marginLog_min_365D',
 'trainer_marginLog_min_91D',
 'trainer_marginLog_std_28D',
 'trainer_marginLog_std_365D',
 'trainer_marginLog_std_91D',
 'trainer_placeLog_max_28D',
 'trainer_placeLog_max_365D',
 'trainer_placeLog_max_91D',
 'trainer_placeLog_mean_28D',
 'trainer_placeLog_mean_365D',
 'trainer_placeLog_mean_91D',
 'trainer_placeLog_median_28D',
 'trainer_placeLog_median_365D',
 'trainer_placeLog_median_91D',
 'trainer_placeLog_min_28D',
 'trainer_placeLog_min_365D',
 'trainer_placeLog_min_91D',
 'trainer_placeLog_std_28D',
 'trainer_placeLog_std_365D',
 'trainer_placeLog_std_91D',
 'trainer_prizemoneyLog_max_28D',
 'trainer_prizemoneyLog_max_365D',
 'trainer_prizemoneyLog_max_91D',
 'trainer_prizemoneyLog_mean_28D',
 'trainer_prizemoneyLog_mean_365D',
 'trainer_prizemoneyLog_mean_91D',
 'trainer_prizemoneyLog_median_28D',
 'trainer_prizemoneyLog_median_365D',
 'trainer_prizemoneyLog_median_91D',
 'trainer_prizemoneyLog_min_28D',
 'trainer_prizemoneyLog_min_365D',
 'trainer_prizemoneyLog_min_91D',
 'trainer_prizemoneyLog_std_28D',
 'trainer_prizemoneyLog_std_365D',
 'trainer_prizemoneyLog_std_91D',
 'trainer_runTimeNorm_max_28D',
 'trainer_runTimeNorm_max_365D',
 'trainer_runTimeNorm_max_91D',
 'trainer_runTimeNorm_mean_28D',
 'trainer_runTimeNorm_mean_365D',
 'trainer_runTimeNorm_mean_91D',
 'trainer_runTimeNorm_median_28D',
 'trainer_runTimeNorm_median_365D',
 'trainer_runTimeNorm_median_91D',
 'trainer_runTimeNorm_min_28D',
 'trainer_runTimeNorm_min_365D',
 'trainer_runTimeNorm_min_91D',
 'trainer_runTimeNorm_std_28D',
 'trainer_runTimeNorm_std_365D',
 'trainer_runTimeNorm_std_91D',
 'dogAgeScaled',
 'lastFiveWinPercentage',
 'lastFivePlacePercentage',
 'weightInKgScaled',
 'rolling_box_win_percentage']

In [11]:
def closure(optimizer, criterion, outs, classes):
    optimizer.zero_grad()
    loss = nn.functional.mse_loss(outs, classes)
    loss.backward()
    return loss

def load_matching_state_dict(model, state_dict):
    model_dict = model.state_dict()

    # Filter out unnecessary keys
    matching_keys = {k: v for k, v in state_dict.items() if k in model_dict and v.size() == model_dict[k].size()}
    excluded_keys = set(state_dict.keys()) - set(matching_keys.keys())

    # Overwrite entries in the existing state dict
    model_dict.update(matching_keys)

    # Load the new state dict
    model.load_state_dict(model_dict)

    print("Loaded keys:")
    for key in matching_keys:
        print(key)

    print("\nExcluded keys:")
    for key in excluded_keys:
        print(key)


def squash_state_dict(model, state_dict):
    model_dict = model.state_dict()

    # Filter out unnecessary keys
    matching_keys = {k: v for k, v in state_dict.items() if k in model_dict and v.size() == model_dict[k].size()}
    excluded_keys = set(state_dict.keys()) - set(matching_keys.keys())

    

    # Overwrite entries in the existing state dict
    model_dict.update(matching_keys)

    # Load the new state dict
    model.load_state_dict(model_dict)

    print("Loaded keys:")
    for key in matching_keys:
        print(key, model_dict[key].size())

    squashed_keys = {}
    print("\nExcluded keys:")
    for key in excluded_keys:

        print(key, model_dict[key].size(), state_dict[key].size())
        try:
            squashed_keys[key] = state_dict[key][model_dict[key].size()[0]:, model_dict[key].size()[1]]
        except Exception as e:
            print('failed', key)

        try:
            squashed_keys[key] = state_dict[key][model_dict[key].size()[0]:]
        except Exception as e:
            print('failed', key)
    print(squashed_keys)
    model_dict.update(squashed_keys)

def model_pipeline(my_dataset=raceDB,config=None,prev_model=None, sweep=True, model_state_dict=None, prev_model_file=None, prev_model_version='450'):
    if my_dataset:
      dataset = my_dataset    
    else:
      dataset = raceDB
    # tell wandb to get started
    with wandb.init(project="NEW GRU V7 Reporting", config=config,save_code=False):
      #  access all HPs through wandb.config, so logging matches execution!
      wandb.define_metric("loss_val", summary="min")
      wandb.define_metric("accuracy", summary="max")
      wandb.define_metric("ROI < 30", summary="max")
      wandb.define_metric("val_ROI < 30", summary="max")
      wandb.define_metric("relu roi", summary="max")
      
      config = wandb.config
      pprint.pprint(config)
      pprint.pprint(config.epochs)
      print(config)
      wandb.run.log_code('rnn_tools/')
      # input_size = raceDB.get_race_input([0,1])[0].full_input.shape[0] #create fix so messy
      print(config.input_type)

      for i in config.stat_list_dict.values():
        print(i)

      reg_stat_mask = []
      for stat,flag in config.stat_list_dict.items():
          stat_flag = flag
          reg_stat_mask = reg_stat_mask+[stat_flag]

      print(reg_stat_mask)
      stat_mask = [1]*6+reg_stat_mask+[0]*80
      # stat_mask = [1]*30
      stat_mask = [1]*18+36*[0]
      data_mask = [0]*26+[1]*20+[1]*20+[1]*20+[1]*20 # Reg, Dist, Box, T_box, T_dist
      data_mask = [1]*6+reg_stat_mask+[0]*20+[0]*20+[0]*20+[0]*20 # Reg, Dist, Box,T_box, T_dist
      data_mask =  [0]*18+36*[1]
      # data_mask =  [1]*30
      data_mask_size = sum(data_mask)
      print(f"{data_mask_size=}")
      print(f"{data_mask=}")
      stat_mask = torch.tensor(stat_mask).type(torch.bool).to(device)
      data_mask = torch.tensor(data_mask).type(torch.bool).to(device)
      # stat_mask = torch.ones_like(stat_mask).type(torch.bool).to(device)
      input_size = sum(stat_mask)

      print(stat_mask)
      print(stat_mask.shape)


      if 'batch_days' in config.keys() and not raceDB.batches_setup:
        pass
        raceDB.create_test_split_date(config['training_date_end'],val_date='2023-07-01')
        raceDB.create_dogs_test_split_date()
        raceDB.attach_races_to_dog_inputv2() 
        raceDB.reset_hidden()
        raceDB.create_batches(batch_days=config['batch_days'], end_date = config['training_date_end'], stat_mask=stat_mask,data_mask=data_mask,gen_packed_seq=True)
        raceDB.batches_setup = True
      if config['input_type'] == 'basic':
          print('here')
          raceDB.batches['packed_x'] = raceDB.batches['packed_x_basic']
          raceDB.batches['packed_y'] = raceDB.batches['packed_y_basic']
          raceDB.batches['packed_v'] = raceDB.batches['packed_v_basic']
          # input_size = raceDB.batches['packed_x'][0].data[0].shape[0]



      print(f"{input_size=}")
      config['stat_mask_tensor'] = torch.tensor(stat_mask, dtype=torch.uint8).to(device)     
      raceDB.reset_hidden(num_layers=config['num_layers'], hidden_size=config['hidden_size'])
      model = rnn_classes.GRUNetv3_extra_embedding(input_size,config['hidden_size'], num_layers=config['num_layers'],fc0_size=config['f0_layer_size'], fc1_size=config['f1_layer_size'],data_mask_size=data_mask_size)
      optimizer = optim.AdamW(model.parameters(), lr=config['learning_rate'])
      
      config['model_name'] = model.__class__.__name__
      if model_state_dict:
        model.load_state_dict(model_state_dict)
      if prev_model_file!=None:
        print(f"Loading model {prev_model_file}, version {prev_model_version}")
        model_name = prev_model_file
        model_loc = f"models/savedmodel/{model_name}/{model_name}_{prev_model_version}.pt"
        model_data = torch.load(model_loc,map_location=torch.device('cuda:0'))
        print(model_data['model_state_dict'].keys())
        # model.load_state_dict(model_data['model_state_dict'], strict=False)
        load_matching_state_dict(model, model_data['model_state_dict'])
        config['parent model'] = prev_model_file
        # raceDB.fill_hidden_states_dict_v2(model_data['db_train'])
        model = model.to(device)
        # raceDB.load_hidden_in_dict(hidden_in_dict=model_data['hidden_ins'],output_dict=model_data['output'])
        
        optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'])
        # optimizer.load_state_dict(model_data['optim'])
      else:
        optimizer = optim.RAdam(model.parameters(), lr=config['learning_rate'])

      raceDB.to_cuda()
      criterion = nn.CrossEntropyLoss(reduction='none')
      wandb.run.name = f"{' '.join(states)}-{wandb.run.name}"
      model = model.to(device)
      print(model)

      try:
        return training_testing_gru_extra_data_embedding.train_double_v3(model, dataset, criterion, optimizer, 'na', config)
      except (KeyboardInterrupt) as error:
        print(error)
        print("finished Early")
        
      raceDB.create_hidden_states_dict_v2()
      model_saver_wandb(model, optimizer, 450, 0.1, raceDB.hidden_states_dict_gru_v6,raceDB.train_hidden_dict , model_name="long nsw new  22000 RUN")


    return (model,dataset, optimizer)

In [12]:
raceDB.batches_setup = False

In [13]:
stat_list_dict = {'_dist_last__1':1,
    '_box_last__1':1,
    '_speed_avg_1':1,
    '_split_speed_avg_1':0,
    '_split_margin_avg_1':1,
    '_margin_avg_1':1,
    '_margin_time_avg_1':1,
    '_RunHomeTime_1':1,
    '_run_home_speed_1':1,
    '_first_out_avg_1':0,
    '_pos_out_avg_1':0,
    '_post_change_avg_1':0,
    '_races_1':1,
    '_wins_1':1,
    '_wins_last_1':1,
    '_weight_':1,
    '_min_time_':1,
    '_min_split_time_':1,
    '_last_start_price':0,
    '_last_start_prob':0,
}

wandb_config_static = {
    'hidden_size': 512,
    'stats': raceDB.stats_cols,
    'races': states,
    'datafile': data_file,
    'latest_date': raceDB.latest_date,
    'input_type': 'basic',
    'num_layers': 2,
    'batch_size': 750,
    'dropout': 0.3,
    'epochs': 2500,
    'learning_rate': 0.0001,
    'optimizer': 'adamW',
    'f0_layer_size': 128,
    'f1_layer_size': 64,
    'training_date_end': '2023-01-01',
    'notes':'GRU, with basic add on data looped in',
    'batch_days': 90,
    'stat_list_dict': stat_list_dict,
}

In [14]:
raceDB.reset_hidden(num_layers=wandb_config_static['num_layers'], hidden_size=wandb_config_static['hidden_size'])
torch.cuda.empty_cache()
raceDB.race_prices_to_prob()
print(raceDB.latest_date)
for race in raceDB.racesDict.values():
    race.loss = torch.tensor(1.0,requires_grad=True,device='cuda:0')  
    race.loss_log = torch.tensor(1.0,requires_grad=True,device='cuda:0')  

for race in raceDB.racesDict.values():
    if race.classes.isnan().sum():
        print(race.classes.sum())
        print(race.raceid)
        race.classes = race.classes.nan_to_num(0,0,0)

2024-04-30 00:00:00


In [57]:
prev_model_file='NZ-stoic-music-566' 
prev_model_version = 410
config = wandb_config_static
model = rnn_classes.GRUNetv3_extra_embedding(18,config['hidden_size'], num_layers=config['num_layers'],fc0_size=config['f0_layer_size'], fc1_size=config['f1_layer_size'],data_mask_size=36)
print(f"Loading model {prev_model_file}, version {prev_model_version}")
model_name = prev_model_file
model_loc = f"models/savedmodel/{model_name}/{model_name}_{prev_model_version}.pt"
model_data = torch.load(model_loc,map_location=torch.device('cuda:0'))
print(model_data['model_state_dict'].keys())
# model.load_state_dict(model_data['model_state_dict'], strict=False)
squash_state_dict(model, model_data['model_state_dict'])
config['parent model'] = prev_model_file

Loading model NZ-stoic-music-566, version 410
odict_keys(['h0', 'gru.weight_ih_l0', 'gru.weight_hh_l0', 'gru.bias_ih_l0', 'gru.bias_hh_l0', 'gru.weight_ih_l1', 'gru.weight_hh_l1', 'gru.bias_ih_l1', 'gru.bias_hh_l1', 'fc0.weight', 'fc0.bias', 'batch_norm.weight', 'batch_norm.bias', 'batch_norm.running_mean', 'batch_norm.running_var', 'batch_norm.num_batches_tracked', 'batch_norm_data.weight', 'batch_norm_data.bias', 'batch_norm_data.running_mean', 'batch_norm_data.running_var', 'batch_norm_data.num_batches_tracked', 'track_embedding.weight', 'extra_1.batch_norm.weight', 'extra_1.batch_norm.bias', 'extra_1.batch_norm.running_mean', 'extra_1.batch_norm.running_var', 'extra_1.batch_norm.num_batches_tracked', 'extra_1.fc0_p1.weight', 'extra_1.fc0_p1.bias', 'extra_1.fc0_p2.weight', 'extra_1.fc0_p2.bias', 'extra_1.fc0_p3.weight', 'extra_1.fc0_p3.bias', 'fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias', 'fc3.weight', 'fc3.bias'])
Loaded keys:
batch_norm.weight torch.Size([18])
batch_norm.bias

In [15]:
importlib.reload(rnn_tools.rnn_classes)
importlib.reload(rnn_classes)
importlib.reload(training_testing_gru_extra_data)
importlib.reload(training_testing_gru_extra_data_embedding)
# importlib.reload(training_testing_gru_extra_data_state_batch)
# importlib.reload(training_testing_gru_extra_data_toasty)
# importlib.reload(training_testing_gru_extra_data_toasty_og)
model_pipeline(raceDB,config=wandb_config_static,sweep=False)
# model_pipeline(raceDB,config=wandb_config_static,sweep=False, prev_model_file='NZ-stoic-music-566', prev_model_version = 410)
# model_pipeline(raceDB,config=wandb_config_static,sweep=False, prev_model_file='NZ-robust-snow-518', prev_model_version = 290)
# model_pipeline(raceDB,config=wandb_config_static,sweep=False, prev_model_file='VIC-zany-cherry-524', prev_model_version = 1025)
# (model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static,sweep=False)
# all_price_df = model_pipeline(raceDB,config=wandb_config_static,sweep=False)

wandb: Currently logged in as: nickojelly. Use `wandb login --relogin` to force relogin


{'hidden_size': 512, 'stats': "['dogAgeScaled', 'boxNumber', 'weightInKgScaled', 'hasEntryBoxNumberPlus1', 'hasEntryBoxNumberMinus1', 'rolling_box_win_percentage', 'dog_distance_mean_1', 'dog_boxNumber_mean_1', 'dog_runTimeNorm_mean_1', 'dog_place_mean_1', 'dog_resultMargin_mean_1', 'dog_split_time_margin_mean_1', 'dog_split_runTimeNorm_mean_1', 'dog_time_1_mean_1', 'dog_run_home_TimeNorm_mean_1', 'dog_finishingPlaceMovement_mean_1', 'dog_averageSpeed_mean_1', 'dog_win_mean_1', 'trainer_distance_mean_365D', 'trainer_boxNumber_mean_365D', 'trainer_runTimeNorm_mean_365D', 'trainer_place_mean_365D', 'trainer_resultMargin_mean_365D', 'trainer_split_time_margin_mean_365D', 'trainer_split_runTimeNorm_mean_365D', 'trainer_time_1_mean_365D', 'trainer_run_home_TimeNorm_mean_365D', 'trainer_finishingPlaceMovement_mean_365D', 'trainer_averageSpeed_mean_365D', 'trainer_win_mean_365D', 'dam_distance_mean_365D', 'dam_boxNumber_mean_365D', 'dam_runTimeNorm_mean_365D', 'dam_place_mean_365D', 'dam_resu

100%|██████████| 2562/2562 [00:00<00:00, 5023.53it/s]


[(datetime.date(2019, 10, 9), datetime.date(2020, 1, 7)), (datetime.date(2020, 1, 7), datetime.date(2020, 4, 6)), (datetime.date(2020, 4, 6), datetime.date(2020, 7, 5)), (datetime.date(2020, 7, 5), datetime.date(2020, 10, 3)), (datetime.date(2020, 10, 3), datetime.date(2021, 1, 1)), (datetime.date(2021, 1, 1), datetime.date(2021, 4, 1)), (datetime.date(2021, 4, 1), datetime.date(2021, 6, 30)), (datetime.date(2021, 6, 30), datetime.date(2021, 9, 28)), (datetime.date(2021, 9, 28), datetime.date(2021, 12, 27)), (datetime.date(2021, 12, 27), datetime.date(2022, 3, 27)), (datetime.date(2022, 3, 27), datetime.date(2022, 6, 25)), (datetime.date(2022, 6, 25), datetime.date(2022, 9, 23)), (datetime.date(2022, 9, 23), datetime.date(2022, 12, 22)), (datetime.date(2022, 12, 22), datetime.date(2022, 12, 31))]
2020-01-07
2020-04-06
2020-07-05
2020-10-03
2021-01-01
2021-04-01
2021-06-30
2021-09-28
2021-12-27
2022-03-27
2022-06-25
2022-09-23
2022-12-22
2022-12-31
Train examples [46, 61, 85, 86, 81, 95

100%|██████████| 2561/2561 [00:00<00:00, 320110.04it/s]
14it [00:00, 23.03it/s]


...

 21%|██        | 520/2500 [1:15:18<10:14:12, 18.61s/it]

Test val pass Time: 0.3119519999891054
val val pass Time: 2.4053907000052277
Test time : 8.175030700018397, Val time : 5.770393199985847, Total time : 13.945423900004243
New Max ROI: 0.2156, -0.0853, 2.08256


 22%|██▏       | 540/2500 [1:21:54<8:06:30, 14.89s/it] 

Test val pass Time: 0.6010675000143237
val val pass Time: 0.6085250999894924
Test time : 4.609239699988393, Val time : 4.2401064000150654, Total time : 8.849346100003459
New Max ROI: 0.2155, -0.0857, 2.08238


 22%|██▏       | 543/2500 [1:22:44<7:58:09, 14.66s/it] 

In [36]:
importlib.reload(rnn_tools.rnn_classes)
importlib.reload(rnn_classes)
importlib.reload(training_testing_gru_extra_data)
# importlib.reload(training_testing_gru_extra_data_state_batch)
# importlib.reload(training_testing_gru_extra_data_toasty)
# importlib.reload(training_testing_gru_extra_data_toasty_og)
(model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static,sweep=False)
# model_pipeline(raceDB,config=wandb_config_static,sweep=False, prev_model_file='NZ-fine-waterfall-435', prev_model_version = 355)
# (model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static,sweep=False)
# all_price_df = model_pipeline(raceDB,config=wandb_config_static,sweep=False)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: nickojelly. Use `wandb login --relogin` to force relogin


{'hidden_size': 256, 'stats': "[['inside', 'midfield', 'wide', 'weight', 'DogGrade', '_dist_last__1', '_box_last__1', '_speed_avg_1', '_split_speed_avg_1', '_split_margin_avg_1', '_margin_avg_1', '_margin_time_avg_1', '_RunHomeTime_1', '_run_home_speed_1', '_first_out_avg_1', '_pos_out_avg_1', '_post_change_avg_1', '_races_1', '_wins_1', '_wins_last_1', '_weight_', '_min_time_', '_min_split_time_', '_last_start_price', '_last_start_prob', 'dist_dist_last__10', 'dist_box_last__10', 'dist_speed_avg_10', 'dist_split_speed_avg_10', 'dist_split_margin_avg_10', 'dist_margin_avg_10', 'dist_margin_time_avg_10', 'dist_RunHomeTime_10', 'dist_run_home_speed_10', 'dist_first_out_avg_10', 'dist_pos_out_avg_10', 'dist_post_change_avg_10', 'dist_races_10', 'dist_wins_10', 'dist_wins_last_10', 'dist_weight_', 'dist_min_time_', 'dist_min_split_time_', 'dist_last_start_price', 'dist_last_start_prob', 'box_dist_last__10', 'box_box_last__10', 'box_speed_avg_10', 'box_split_speed_avg_10', 'box_split_margin

100%|██████████| 14184/14184 [00:09<00:00, 1542.92it/s]


[(datetime.date(2019, 12, 1), datetime.date(2020, 12, 25)), (datetime.date(2020, 12, 25), datetime.date(2022, 1, 19)), (datetime.date(2022, 1, 19), datetime.date(2022, 12, 31))]
2020-12-25
2022-01-19
2022-12-31
Train examples [1, 8267, 12511]


100%|██████████| 14183/14183 [00:18<00:00, 784.27it/s]
3it [00:30, 10.04s/it]


Train examples [8, 5853, 7341]
Train examples [8, 5853, 7341]
Train examples [1, 8267, 12511]
Train examples [1, 8267, 12511]
here
input_size=tensor(305, device='cuda:0')


C:\Users\Nick\AppData\Local\Temp\ipykernel_16964\2788016069.py:72: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  config['stat_mask_tensor'] = torch.tensor(stat_mask, dtype=torch.uint8).to(device)
c:\Users\Nick\.conda\envs\python311\Lib\site-packages\torch\nn\modules\lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


GRUNetv3_extra(
  (gru): GRU(305, 256, num_layers=2, dropout=0.3)
  (relu): ReLU()
  (fc0): Linear(in_features=2630, out_features=2048, bias=True)
  (batch_norm): BatchNorm1d(305, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batch_norm_data): BatchNorm1d(305, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (extra_1): GRUNetv3_simple_extra_data(
    (batch_norm): LazyBatchNorm1d(0, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
    (fc0_p1): LazyLinear(in_features=0, out_features=128, bias=True)
    (fc0_p1_drop): Dropout(p=0.3, inplace=False)
    (fc0_p2): Linear(in_features=128, out_features=64, bias=True)
    (fc0_p2_drop): Dropout(p=0.3, inplace=False)
    (fc0_p3): Linear(in_features=64, out_features=64, bias=True)
    (fc0_p3_drop): Dropout(p=0.3, inplace=False)
  )
  (relu0): ReLU()
  (drop1): Dropout(p=0.3, inplace=False)
  (fc1): Linear(in_features=2048, out_features=1024, bias=True)
  (drop2): Dropout(

  0%|          | 0/2500 [00:00<?, ?it/s]

In [ ]:
profit_model = rnn_classes.GRUNetv3_profit(raceDB).to('cuda:0')

In [ ]:
raceDB.states

In [ ]:
new_batches = []
for batch in batch_races:
    batch_dict = {state:[] for state in raceDB.states}
    for race in batch:
        batch_dict[race.state].append(race)
    new_batch = list(batch_dict.values())
    new_batches.append(new_batch)
    for k,v in batch_dict.items():
        print(f"{k} races {len(v)}")
raceDB.batches['new_batch_races'] = new_batches

In [ ]:
[print(len(batch)) for batch in raceDB.batches['new_batch_races']]

In [ ]:
list(batch_dict.values())

In [ ]:
importlib.reload(rnn_tools.rnn_classes)
importlib.reload(rnn_classes)
importlib.reload(training_testing_gru_extra_data)
(model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static,sweep=False)

# profit_tensor =  model_pipeline(raceDB,config=wandb_config_static,sweep=False)

In [ ]:
def print_grad_fn(tensor):
    print(tensor)
    if tensor is not None:
        if tensor is not None:
            for t in tensor.next_functions:
                if t[0] is not None:
                    # print(f"{t[0]=}")
                    print_grad_fn(t[0])

# Usage:
# Assuming `output` is your tensor
# print_grad_fn(output)

In [ ]:
profit_tensor.grad_fn.next_functions

In [ ]:
profit_tensor.shape

In [ ]:
print_grad_fn(profit_tensor[0].grad_fn)

In [ ]:
profit_tensor.grad_fn.next_functions

In [ ]:
len_test = len(raceDB.test_race_ids)
test_idx = range(0,len_test)
race = raceDB.get_test_input(test_idx)

In [ ]:
for r in race:
    print(r.prices)
    print(r.raceid)
    _, actual = torch.max(r.classes, 0)
    onehot_win = F.one_hot(actual, num_classes=8)
    print(onehot_win)
    print(r.win_price_weight)
    print(r.raw_margins)
    print(r.race_date)
    print(r.track_name)
    print('----------------------------')

In [ ]:
import plotly.graph_objs as go
import plotly.subplots as sp

hidden_outs = []
margins = []
for race in raceDB.dogsDict['410839665'].races.values():
    hidden_outs.append(race.hidden_out.detach())
    margins.append(race.margin)
# Assuming hidden_states is your list of tensors
hidden_states = [x[0:256] for x in hidden_outs]
hidden_states = [hidden_state.reshape(16, 16) for hidden_state in hidden_states]
# Create a heatmap for each step
frames = [go.Frame(
    data=[go.Heatmap(z=hidden_state.tolist(), colorscale='Viridis')],
    name=str(i)
) for i, hidden_state in enumerate(hidden_states)]

# Create a slider
sliders = [dict(
    steps=[dict(method='animate',
                args=[[frame['name']]],
                label=str(i)) for i, frame in enumerate(frames)],
    transition=dict(duration=300, easing='cubic-in-out'),
    active=0,
)]

# Create a layout
layout = go.Layout(
    sliders=sliders,
    updatemenus=[dict(type='buttons',
                      showactive=False,
                      buttons=[dict(label='Play',
                                    method='animate',
                                    args=[None, {"frame": {"duration": 500, "redraw": False},
                                                 "fromcurrent": True,
                                                 "transition": {"duration": 300,
                                                                "easing": "quadratic-in-out"}}]),
                                  dict(label='Stop',
                                       method='animate',
                                       args=[[None], {"frame": {"duration": 0, "redraw": False},
                                                      "mode": "immediate",
                                                      "transition": {"duration": 0}}])])],
)

# Create a figure
fig = go.Figure(data=frames[0]['data'], layout=layout, frames=frames)

# Display the figure
fig.show()

In [ ]:
import plotly.graph_objs as go
import plotly.subplots as sp
from plotly.subplots import make_subplots
hidden_inits= [hidden_state[1,:].reshape(16, 16) for hidden_state in raceDB.hidden_state_inits]

dogs = ['366590730','410839665']

hiddens = []
margins = []
for dog in dogs:
    hidden_outs = []
    margin = []
    for race in raceDB.dogsDict[dog].races.values():

        hidden_outs.append(race.hidden_out.detach())
        margin.append(race.margin)
    hiddens.append(hidden_outs)
    margins.append(margin)
    
min_len = min([len(x) for x in hiddens])
print(f"{min_len=}")
hidden_states = [[hidden_state[0:256].cpu().numpy().reshape(16, 16) for hidden_state in dog[0:min_len]] for dog in hiddens]
print(f"{[len(x) for x in hidden_states]=}")
# Create a heatmap for each step
global_min = min(hidden_state.min() for hidden_state in hidden_states[0])
global_max = max(hidden_state.max() for hidden_state in hidden_states[0])

print(global_min, global_max)
# print(hidden_states[0])
# Create a heatmap for each step with constant scale
# Create a subplot for each dog
# Create a subplot for each dog with an additional row for the line plot
fig = make_subplots(rows=4, cols=1)

# Create a heatmap and line plot for each dog
for i, dog_hidden_states in enumerate(hidden_states):
    # Create heatmap
    heatmap = go.Heatmap(z=dog_hidden_states[0].tolist(), zmin=global_min, zmax=global_max, colorscale='Viridis')
    fig.add_trace(heatmap, row=2*i+2, col=1)

    # Create line plot
    line_plot = go.Scatter(y=[sum(abs(state.flatten())) for state in dog_hidden_states], mode='lines',showlegend=False)
    fig.add_trace(line_plot, row=2*i+1, col=1)

frames = []

# Iterate over dogs and their corresponding hidden states
for j in range(min_len):
    frame_data = []
    for i, dog_hidden_states in enumerate(hidden_states):
        hidden_state = dog_hidden_states[j]
        heatmap = go.Heatmap(z=hidden_state.tolist(), zmin=global_min, zmax=global_max, colorscale='Viridis')
        line_plot = go.Scatter(y=[sum(abs(state.flatten())) for state in dog_hidden_states[:j+1]], mode='lines',showlegend=False)
        frame_data.extend([heatmap, line_plot])
    frame = go.Frame(data=frame_data, name=str(j))  # Create a frame for each time step
    frames.append(frame)

# Define the slider
# Define the slider
sliders = [dict(steps=[dict(method='animate',
                            args=[[f.name], {"mode": "immediate",
                                             "frame": {"duration": 100, "redraw": True},
                                             "transition": {"duration": 0}}],
                            label=f.name) for f in frames],
                active=0)]

# Update layout
fig.update_layout(
    updatemenus=[dict(type='buttons',
                      showactive=False,
                      buttons=[dict(label='Play',
                                    method='animate',
                                    args=[None, {"frame": {"duration": 100, "redraw": False},
                                                 "fromcurrent": True,
                                                 "mode": "immediate",
                                                 "transition": {"duration": 0,
                                                                "easing": "quadratic-in-out"}}]),
                                  dict(label='Stop',
                                       method='animate',
                                       args=[[None], {"frame": {"duration": 0, "redraw": False},
                                                      "mode": "immediate",
                                                      "transition": {"duration": 0}}])])],
    sliders=sliders
)
# Update frames
fig.frames = frames

# Display the figure
fig.show()
import plotly.io as pio

# Save the figure as an interactive html file
pio.write_html(fig, 'hidden_simple_double.html')

In [ ]:
# Create a line plot for each dog
fig = go.Figure()

for i, dog_hidden_states in enumerate(hidden_states):
    # Calculate the absolute sum of hidden states for each time step
    abs_sum = [sum(abs(state.flatten())) for state in dog_hidden_states]
    print(abs_sum)

    # Create line plot with x values specified
    line_plot = go.Scatter(x=list(range(len(dog_hidden_states))), y=abs_sum, mode='lines', name=f'Dog {i+1}')
    fig.add_trace(line_plot)

# Update layout
fig.update_layout(
    title="Absolute Sum of Hidden States Over Time",
    xaxis_title="Time Step",
    yaxis_title="Absolute Sum of Hidden States",
    legend_title="Dogs",
)

# Display the figure
fig.show()

In [ ]:
from bokeh.layouts import column
from bokeh.models import Slider, CustomJS
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.io import curdoc
import numpy as np

# Assuming hidden_states is a list of 2D numpy arrays
hidden_states = [[hidden_state[0:256].cpu().numpy().reshape(16, 16) for hidden_state in dog[0:min_len]] for dog in hiddens]

# Create a ColumnDataSource to hold the data for the current frame
source = ColumnDataSource(data={'image': [hidden_states[0][0]]})

# Create a figure and add an image renderer
p = figure(x_range=(0, 1), y_range=(0, 1), width=400, height=400, match_aspect=True)
p.image(image='image', x=0, y=0, dw=1, dh=1, source=source)

# Create a slider that goes from the first to the last frame
slider = Slider(start=0, end=len(hidden_states[0])-1, value=0, step=1)

# Define a callback function to update the ColumnDataSource's data when the slider is changed
callback = CustomJS(args=dict(source=source, states=hidden_states[0]), code="""
    var i = cb_obj.value;
    source.data = {'image': [states[i]]};
""")

# Attach the callback to the slider
slider.js_on_change('value', callback)

# Display the figure and slider
layout = column(p, slider)
show(layout)

In [ ]:
for i in raceDB.hidden_state_inits:
    print(i[0,0])

In [ ]:
hidden_state_init = model.h0
raceDB.reset_hidden_w_param(hidden_state_init,num_layers=2, hidden_size=256)

In [ ]:
raceDB.dogsDict['408750377'].hidden